In [0]:
dbutils.widgets.text('start', '')
dbutils.widgets.text('end', '')
dbutils.widgets.text('vendor', '')
dbutils.widgets.text('marketplace', '')

start = dbutils.widgets.get('start')
end = dbutils.widgets.get('end')
vendor = dbutils.widgets.get('vendor')
marketplace = dbutils.widgets.get('marketplace')


# IAM User Credentials , IAM Role 
A_KEY = 'AKIA.........UGXH2GE'
A_SECRET = 'Kz54.........x'
ARN = 'arn:aws:iam::46....17:role/sellingapiRole'



if marketplace == 'DE':
    MarketplaceID = 'A1PA6795UKMFR9'
elif marketplace == 'FR':
    MarketplaceID = 'A13V1IB3VIYZZH'
elif marketplace == 'ES':
    MarketplaceID = 'A1RKKUPIHCS9HS'
elif marketplace == 'UK':
    MarketplaceID = 'A1F83G8C2ARO7P'
elif marketplace == 'IT':
    MarketplaceID = 'APJ6JRA9NG5V4'
elif marketplace == 'NL':
    MarketplaceID = 'A1805IZSGTT6HS'
elif marketplace == 'PL':
    MarketplaceID = 'A1C3SOZRARQ6R3'
elif marketplace == 'SE':
    MarketplaceID = 'A2NODRKZP88ZB9'


# Amazon Vendor Central App Client credentials based on Marketplace and Vendor Account    
if vendor == 'NP' and marketplace == 'DE':
    C_KEY = 'amzn1.application-oa2-client............a3eb46e5d'
    C_SECRET_KEY = 'a22637...................1e9da2d'
    C_REFRESH_TOKEN = 'Atzr|IwEx03-46D.....................5KIJ1QFPIXK_5HCHU'


elif vendor == 'SW' and marketplace == 'FR':
    C_KEY = 'amzn1.application-oa2-client.......f5110b16'
    C_SECRET_KEY = '927933fbfe01f332.........6359d10a9629'
    C_REFRESH_TOKEN = 'Atzr|IwEBILtbEPHrTcQ.M........RyYOmAVKbp9HhrM'
  

In [0]:
from time import sleep
import requests
import re
import json
import datetime
import gzip
from requests_aws4auth import AWS4Auth



def get_access_token():
    global C_KEY, C_SECRET_KEY, C_REFRESH_TOKEN
    r = requests.post(f"https://api.amazon.com/auth/o2/token", data={ "grant_type":"refresh_token",
                                                                      "refresh_token":C_REFRESH_TOKEN,
                                                                      "client_id": C_KEY,
                                                                      "client_secret": C_SECRET_KEY } )
    return r.json()['access_token']



def assume_role():
    r = requests.get(f"https://sts.amazonaws.com?Version=2011-06-15&Action=AssumeRole&RoleSessionName=Test&RoleArn={ARN}&DurationSeconds=3600", 
                     auth = AWS4Auth(A_KEY, A_SECRET, 'us-east-1', 'sts'))
    
    return (re.findall('<AccessKeyId>(.+)</AccessKeyId>', r.text)[0], re.findall('<SecretAccessKey>(.+)</SecretAccessKey>', r.text)[0], re.findall('<SessionToken>(.+)</SessionToken>', r.text)[0])



# get the keys
access_token = get_access_token()
access_key, secret_key, session_token = assume_role()



def get_request(rel_url):
    global access_token, access_key, secret_key, session_token

    r = requests.get(f"https://sellingpartnerapi-eu.amazon.com{rel_url}", 
                     headers={'x-amz-access-token': access_token},
                     auth = AWS4Auth(access_key, secret_key, 'eu-west-1', 'execute-api', session_token=session_token))
    return(r.json())

    


def post_request(rel_url, data):
    global access_token, access_key, secret_key, session_token

    r = requests.post(f"https://sellingpartnerapi-eu.amazon.com{rel_url}",
        data=json.dumps(data),
        headers = { 'x-amz-access-token': access_token,
                    'Content-Type': 'application/json'
                  },
        auth = AWS4Auth(access_key, secret_key, 'eu-west-1', 'execute-api', session_token=session_token))
    
    return (r.json())




def generate_report(report_type, start, end, distributor_type):
    req = post_request('/reports/2021-06-30/reports', { "marketplaceIds": [MarketplaceID],
                                                        'reportType': report_type,
                                                        'dataStartTime': start,
                                                        'dataEndTime': end,
                                                        "reportOptions": { "reportPeriod": "DAY" ,
                                                                           "sellingProgram": "RETAIL",
                                                                           "distributorView": distributor_type }
                                                       }
                       )
    print(req)
    return req['reportId']




def download_report(report_id):
    gr = get_request(f'/reports/2021-06-30/reports/{report_id}')
    document_id = gr['reportDocumentId']
    gd = get_request(f'/reports/2021-06-30/documents/{document_id}')
    url = gd['url']
    req = requests.get(url)
    zipfile = req.content
    file = (gzip.decompress(zipfile).decode('utf-8'))
    return file


In [0]:
containerName = "amazon-vendors-api-all"
storageAccountName = "blobsaless"
sas = "sp=racwdl.........b37e9s%3D"
config = "fs.azure.sas." + containerName+ "." + storageAccountName + ".blob.core.windows.net"

try:
    dbutils.fs.unmount("/mnt/vendor-data-api-all")
except:
    pass
    
try:
    dbutils.fs.mount(
      source = f"wasbs://{containerName}@{storageAccountName}.blob.core.windows.net",
      mount_point = "/mnt/vendor-data-api-all",
      extra_configs = {config : sas})
except:
    pass

In [0]:
def gen_down_save(vendor, start, end, distributor_type):
    report_id = generate_report('GET_VENDOR_SALES_REPORT', start, end, distributor_type)
    print(report_id)
    sleep(1300)
    jsonFile = download_report(report_id)
    dbutils.fs.put('dbfs:/mnt/vendor-data-api-all/' + vendor + '__' + start.replace(":", "_") + '__' + end.replace(":", "_") + '__' + distributor_type + f'__{marketplace}' '.json', jsonFile, overwrite = True)

    

gen_down_save(vendor, start, end, "MANUFACTURING")
gen_down_save(vendor, start, end, "SOURCING")

{'reportId': '50479019248'}
50479019248